In [43]:
# Sergio Lopez Urzaiz
# A00827462

In [44]:
# Librerias necesarias
import agentpy as ap
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random
import datetime

In [45]:
class Almacen(ap.Model):

    def setup(self):

        # Generamos variables para tiempo
        self.timeStart = datetime.datetime.now()

        # Obtenemos numero de robots
        n_robots = int(self.p.robots)
        # Obtenemos numero de cajas
        n_cajas = int(self.p.cajas)

        # Creamos unidad de metro
        metro = self.agents = ap.AgentList(self, int(self.p.size)**2)

        # Creamos grid de almacen
        self.suelo = ap.Grid(self, [self.p.size]*2, track_empty=True)
        self.suelo.add_agents(metro, random=False, empty=True)

        # Damos color de suelo
        self.suelo.agents[0:10, 0:10].condition = 0

        # Asigniamos estantes
        self.suelo.agents[0, 0].condition = 1
        self.suelo.agents[0, int(self.p.size) - 1].condition = 1
        self.suelo.agents[int(self.p.size) - 1, 0].condition = 1
        self.suelo.agents[int(self.p.size) - 1, int(self.p.size) - 1].condition = 1

        # Creamos variables de movimientos
        self.moves = 0

        # Guardamos numero de cajas
        self.cajas = int(self.p.cajas)

        # Creamos lista de robots
        self.listaRobots = []

        # Asignamos robots
        self.suelo.agents[0,1].condition = 2
        self.suelo.agents[0,int(self.p.size)-2].condition = 2
        self.suelo.agents[int(self.p.size)-1, 1].condition = 2
        self.suelo.agents[int(self.p.size)-1, int(self.p.size)-2].condition = 2

        # Creamos lista de cajas
        self.listaCajas = []

        # Asignamos cajas
        for i in range(n_cajas):
            x = random.randint(2,7)
            y = random.randint(2,7)
            self.suelo.agents[x, y].condition = 3

    def step(self):

        # Variable con la que veremos si entrego caja
        delivered = False

        # Checamos si existe algun robot que pueda recoger una caja
        for agent in self.suelo.agents:
            # Encontramos robot que no tiene caja
            if agent.condition == 2:
                for neighbor in self.suelo.neighbors(agent):
                    if neighbor.condition == 3:
                        agent.condition = 4
                        neighbor.condition = 0
                        break
            # Encontramos robot que tiene caja
            if agent.condition == 4:
                delivered = False
                for neighbor in self.suelo.neighbors(agent):
                    if neighbor.condition == 1:
                        agent.condition = 2
                        delivered = True
                        print('Caja entregada')
                        self.cajas -= 1
                        if(self.cajas == 0):
                            print('Movimientos necesitados:', self.moves)
                            self.timeStart = datetime.datetime.now() - self.timeStart
                            time = self.timeStart.seconds + self.timeStart.microseconds/1000000
                            print("Tomo:",time, "segundos")
                        break
                # Si no entrega paquete, se mueve hacia estante
                if delivered == False:
                    dict_pos = self.suelo.positions
                    # Obtenemos coordenadas de nuestro agente
                    x = dict_pos[agent][0]
                    y = dict_pos[agent][1]
                    # Checamos a que estante esta mas cerca
                    up = False
                    down = False
                    left = False
                    right = False
                    # Arriba
                    if x <= int(self.p.size)/2:
                        up = True
                    else:
                        down = True
                    if y <= int(self.p.size)/2:
                        left = True
                    else:
                        right = True
                    # Sabiendo su posicion, lo movemos al azar con una variable
                    if(up and left):
                        move = random.randint(1,2)
                        # Izquierda
                        if move == 1:
                            if(y > 0):
                                if(self.suelo.agents[x][y-1].condition == 0):
                                    agent.condition = 0
                                    self.suelo.agents[x][y-1].condition = 4
                                    self.moves += 1
                        # Arriba
                        if move == 2:
                            if(x > 0):
                                if(self.suelo.agents[x-1][y].condition == 0):
                                    agent.condition = 0
                                    self.suelo.agents[x-1][y].condition = 4
                                    self.moves += 1
                    if(up and right):
                        move = random.randint(1,2)
                        # Derecha
                        if move == 1:
                            if(y < int(self.p.size) - 1):
                                if(self.suelo.agents[x][y+1].condition == 0):
                                    agent.condition = 0
                                    self.suelo.agents[x][y+1].condition = 4
                                    self.moves += 1
                        # Arrirba
                        if move == 2:
                            if(x > 0):
                                if(self.suelo.agents[x-1][y].condition == 0):
                                    agent.condition = 0
                                    self.suelo.agents[x-1][y].condition = 4
                                    self.moves += 1
                    if(down and left):
                        move = random.randint(1,2)
                        # Izquierda
                        if move == 1:
                            if(y > 0):
                                if(self.suelo.agents[x][y-1].condition == 0):
                                    agent.condition = 0
                                    self.suelo.agents[x][y-1].condition = 4
                                    self.moves += 1
                        # Abajo
                        if move == 2:
                            if(x < int(self.p.size) - 1):
                                if(self.suelo.agents[x+1][y].condition == 0):
                                    agent.condition = 0
                                    self.suelo.agents[x+1][y].condition = 4
                                    self.moves += 1
                    if(down and right):
                        move = random.randint(1,2)
                        # Derecha
                        if move == 1:
                            if(y < int(self.p.size)-1):
                                if(self.suelo.agents[x][y+1].condition == 0):
                                    agent.condition = 0
                                    self.suelo.agents[x][y+1].condition = 4
                                    self.moves += 1
                        # Abajo
                        if move == 2:
                            if(x < int(self.p.size)-1):
                                if(self.suelo.agents[x+1][y].condition == 0):
                                    agent.condition = 0
                                    self.suelo.agents[x+1][y].condition = 4
                                    self.moves += 1
            # Sigue sin tener caja el robot
            elif agent.condition == 2:
                # Creamos un for que recorre los vecinos y que se mueve de manera al azar
                for neighbor in self.suelo.neighbors(agent):
                    move = random.randint(1,8)
                    if move == 5 and neighbor.condition == 0:
                        agent.condition = 0
                        neighbor.condition = 2
                        self.moves += 1
                        break
    def end(self):
        pass

In [46]:
parameters = {'cajas': 4,
              'steps': 1000,
              'robots': 4,
              'size': 10
}

In [47]:
def animation_plot(model, ax):
    attr_grid = model.suelo.attr_grid('condition')
    color_dict = {0:'#E1C699', 1: '#000080', 2: '#FF0000', 3: '#964B00', 4: '#FF0000'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulacion de almacen\n")

In [48]:
fig, ax = plt.subplots()
model = Almacen(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

Caja entregada
Caja entregada
Caja entregada
